In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os

In [0]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark

In [0]:
findspark.init()
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

GETTING THE DATA FROM GOOGLE DRIVE


In [0]:
#getting the files from drive
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded1 = drive.CreateFile({'id':'1ukV6j1t_Yfmsn4zW_yGD50Uf2Kedp73d'})
downloaded.GetContentFile('com_pre_final.csv') 

In [0]:
downloaded2= drive.CreateFile({'id':'1XoXuDkK0-t0mAru9Pimtmb06_LzyIhv6'})
downloaded2.GetContentFile('combined_data.csv') 

IMPORTING LIBRARIES

In [0]:
#get the dataset
dataset = spark.read.csv('com_pre_final.csv',inferSchema=True, header =True)
v_dataset = spark.read.csv('combined_data.csv', inferSchema = True , header = True)

In [0]:
v_dataset.show(truncate = False)

+---+----------------+------+-------+----------------+-------------+---------+------------+----------------+-------------+---------+-----+------------+----------+------------+-----------+----------------+----------+----------+----+-----+----+----+
|_c0|Type of Customer|Region|S.Offie|Regioal S Office|District Code|City Code|Cluster Code|Distributor Code|Customer Code|WareHouse|STLOC|Product Code|Ref       |Invoice Date|Transporter|Delivery Challan|Del Date  |Truck Code|MODE|Dist |QTY |MRP |
+---+----------------+------+-------+----------------+-------------+---------+------------+----------------+-------------+---------+-----+------------+----------+------------+-----------+----------------+----------+----------+----+-----+----+----+
|0  |DC              |CT    |N4B    |D20             |867          |C004     |ZNR         |9124128125      |9122285036   |E06      |B132 |1790        |6825514790|01.01.2017  |916001456  |5306834285      |29-12-2016|CG04HX7775|Road|299.0|23.0|0.0 |
|1  |DC 

In [0]:
v_dataset = v_dataset.drop('_c0')

In [0]:
# seeing the data types of each column
v_dataset.printSchema()

root
 |-- Type of Customer: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- S.Offie: string (nullable = true)
 |-- Regioal S Office: string (nullable = true)
 |-- District Code: integer (nullable = true)
 |-- City Code: string (nullable = true)
 |-- Cluster Code: string (nullable = true)
 |-- Distributor Code: long (nullable = true)
 |-- Customer Code: long (nullable = true)
 |-- WareHouse: string (nullable = true)
 |-- STLOC: string (nullable = true)
 |-- Product Code: long (nullable = true)
 |-- Ref: long (nullable = true)
 |-- Invoice Date: string (nullable = true)
 |-- Transporter: integer (nullable = true)
 |-- Delivery Challan: long (nullable = true)
 |-- Del Date: string (nullable = true)
 |-- Truck Code: string (nullable = true)
 |-- MODE: string (nullable = true)
 |-- Dist: string (nullable = true)
 |-- QTY: string (nullable = true)
 |-- MRP: double (nullable = true)



In [0]:
#Renaming the colnames
v_dataset= v_dataset.withColumnRenamed("Type of Customer", "Type_of_Customer").withColumnRenamed("S.Offie", "SOffie").withColumnRenamed("Regioal S Office", "Regioal_S_Office").withColumnRenamed("District Code", "District_Code").withColumnRenamed("City Code", "City_Code").withColumnRenamed("Cluster Code", "Cluster_Code").withColumnRenamed("Distributor Code", "Distributor_Code").withColumnRenamed("Customer Code", "Customer_Code").withColumnRenamed("Product Code", "Product_Code").withColumnRenamed("Invoice Date", "Invoice_Date").withColumnRenamed("Del Date", "Del_Date").withColumnRenamed("Truck Code", "Truck_Code")

In [0]:
#Checking how many rows are there in the dataframe
v_dataset.count()

2291856

In [0]:
#selecting particular columns
v_dataset.select('Region','Product_Code','QTY').show()

+------+------------+----+
|Region|Product_Code| QTY|
+------+------------+----+
|    CT|        1790|23.0|
|    HP|        1805|10.0|
|    WB|        1790|16.0|
|    HP|        1788|10.0|
|    WB|        1788|16.0|
|    WB|        1788|21.0|
|    HP|        1805|10.0|
|    HP|        1805|10.0|
|    HP|        1805|20.0|
|    HP|        1805|20.0|
|    HP|        1805|10.0|
|    HP|        1805|10.0|
|    HP|        1805|10.0|
|    HP|        1788|10.0|
|    HP|        1805|20.0|
|    HP|        1805|10.0|
|    CT|        1788|18.0|
|    GJ|        1792|12.0|
|    GJ|        9813| 5.0|
|    GJ|        1790|22.0|
+------+------------+----+
only showing top 20 rows



In [0]:
#Getting the summary of a column
v_dataset.describe().show()

+-------+----------------+-------+-------+----------------+-----------------+---------+------------------+-----------------+-------------------+---------+-------+--------------------+-------------------+------------+-------------------+--------------------+----------+--------------------+-------+------------------+-----------------+------------------+
|summary|Type_of_Customer| Region| SOffie|Regioal_S_Office|    District_Code|City_Code|      Cluster_Code| Distributor_Code|      Customer_Code|WareHouse|  STLOC|        Product_Code|                Ref|Invoice_Date|        Transporter|    Delivery Challan|  Del_Date|          Truck_Code|   MODE|              Dist|              QTY|               MRP|
+-------+----------------+-------+-------+----------------+-----------------+---------+------------------+-----------------+-------------------+---------+-------+--------------------+-------------------+------------+-------------------+--------------------+----------+--------------------+---

In [0]:
#filter -application
v_dataset.filter(v_dataset.QTY == '23').show()

+----------------+------+------+----------------+-------------+---------+------------+----------------+-------------+---------+-----+------------+----------+------------+-----------+----------------+----------+----------+----+-----+---+----+
|Type_of_Customer|Region|SOffie|Regioal_S_Office|District_Code|City_Code|Cluster_Code|Distributor_Code|Customer_Code|WareHouse|STLOC|Product_Code|       Ref|Invoice_Date|Transporter|Delivery Challan|  Del_Date|Truck_Code|MODE| Dist|QTY| MRP|
+----------------+------+------+----------------+-------------+---------+------------+----------------+-------------+---------+-----+------------+----------+------------+-----------+----------------+----------+----------+----+-----+---+----+
|              DC|    CT|   N4F|             D20|          246|     C206|         ZNR|      9124157951|   9122306886|      E06| B132|        1790|6825904496|  05.03.2017|  916002879|      5306993917|04-03-2017|CG04JD9187|Road|203.0| 23| 0.0|
|              DE|    CT|   N4F|

In [0]:
#Showing unique values for particular columns
v_dataset.select('Region','Product_Code').distinct().show()

+------+------------+
|Region|Product_Code|
+------+------------+
|    WB|        1784|
|    TG|        1790|
|    KL|        1788|
|    PB|        1790|
|    DD|        9813|
|    DL|        1799|
|    WB|        1797|
|    RJ|        1790|
|    DL|        1790|
|    CH|        1790|
|    CT|        3079|
|    CH|        9786|
|    AR|        3079|
|    TG|        3297|
|    TG|        3079|
|    MH|        1784|
|    KL|        1821|
|    KA|        6869|
|    JK|        1790|
|    KA|        1796|
+------+------------+
only showing top 20 rows



In [0]:
v_dataset.filter(v_dataset.Type_of_Customer == 'IC').show()

+----------------+------+------+----------------+-------------+---------+------------+----------------+-------------+---------+-----+------------+----------+------------+-----------+----------------+----------+----------+----+----+-----+---+
|Type_of_Customer|Region|SOffie|Regioal_S_Office|District_Code|City_Code|Cluster_Code|Distributor_Code|Customer_Code|WareHouse|STLOC|Product_Code|       Ref|Invoice_Date|Transporter|Delivery Challan|  Del_Date|Truck_Code|MODE|Dist|  QTY|MRP|
+----------------+------+------+----------------+-------------+---------+------------+----------------+-------------+---------+-----+------------+----------+------------+-----------+----------------+----------+----------+----+----+-----+---+
|              IC|    WB|   N3C|             D19|          802|     WM59|         ZNR|      9121002001|   9122196150|      E13| SK06|        1797|6825687112|  01.02.2017|  916002812|      5306910354|31-01-2017| WB25D5885|Road|47.0|15.01|0.0|
|              IC|    WB|   N3C|

In [0]:
#multiple conditions
v_dataset.where((v_dataset.Regioal_S_Office == 'D23') & (v_dataset.Product_Code == '1805')).show()

+----------------+------+------+----------------+-------------+---------+------------+----------------+-------------+---------+-----+------------+----------+------------+-----------+----------------+----------+----------+----+-----+----+---+
|Type_of_Customer|Region|SOffie|Regioal_S_Office|District_Code|City_Code|Cluster_Code|Distributor_Code|Customer_Code|WareHouse|STLOC|Product_Code|       Ref|Invoice_Date|Transporter|Delivery Challan|  Del_Date|Truck_Code|MODE| Dist| QTY|MRP|
+----------------+------+------+----------------+-------------+---------+------------+----------------+-------------+---------+-----+------------+----------+------------+-----------+----------------+----------+----------+----+-----+----+---+
|              DC|    HP|   N1A|             D23|          273|     H341|         ZNR|      9124135005|   9122193944|      E04| Z115|        1805|6825515371|  01.01.2017|  916001137|      5306837889|30-12-2016| HP64B3365|Road|122.0|10.0|0.0|
|              DC|    HP|   N1A|

In [0]:
#Suppose you want to know the no of occurences of each value in a column
QTY_df = v_dataset.groupby('QTY').count().show()

+--------+-----+
|     QTY|count|
+--------+-----+
|     8.5| 6786|
|   20.64|  211|
|   19.06|  134|
|    10.7|   32|
|   13.87|    9|
|1,020.50|    1|
|   23.97|   29|
|   28.79|    4|
|   31.47|    4|
|   26.63|    5|
|   28.37|    9|
|   37.21|    1|
|   36.65|    1|
|   34.71|    6|
|   34.63|    3|
|   17.42|    1|
|   151.0|    1|
|   30.86|    1|
|     1.0| 5608|
|    20.5|  409|
+--------+-----+
only showing top 20 rows



In [0]:
v_dataset.orderBy(v_dataset.MRP).show()

+----------------+------+------+----------------+-------------+---------+------------+----------------+-------------+---------+-----+------------+----------+------------+-----------+----------------+----------+-----------+----+-----+----+---+
|Type_of_Customer|Region|SOffie|Regioal_S_Office|District_Code|City_Code|Cluster_Code|Distributor_Code|Customer_Code|WareHouse|STLOC|Product_Code|       Ref|Invoice_Date|Transporter|Delivery Challan|  Del_Date| Truck_Code|MODE| Dist| QTY|MRP|
+----------------+------+------+----------------+-------------+---------+------------+----------------+-------------+---------+-----+------------+----------+------------+-----------+----------------+----------+-----------+----+-----+----+---+
|              DC|    JH|   N5B|             D21|          332|     J309|         ZNR|      9124158632|   9122308903|      GI5| B1L1|        1799|6826382245|  17.05.2017|  910014247|      5360736206|17-05-2017|JH01AV-9350|Road| 45.0|20.0|0.0|
|              DC|    SK|   

In [0]:
#If u want to sort the dataframe by any column
sorte_df = v_dataset.sort(dataset.QTY).show()

+----------------+------+------+----------------+-------------+---------+------------+----------------+-------------+---------+-----+------------+----------+------------+-----------+----------------+----------+-----------+----+----+-----+----+
|Type_of_Customer|Region|SOffie|Regioal_S_Office|District_Code|City_Code|Cluster_Code|Distributor_Code|Customer_Code|WareHouse|STLOC|Product_Code|       Ref|Invoice_Date|Transporter|Delivery Challan|  Del_Date| Truck_Code|MODE|Dist|  QTY| MRP|
+----------------+------+------+----------------+-------------+---------+------------+----------------+-------------+---------+-----+------------+----------+------------+-----------+----------------+----------+-----------+----+----+-----+----+
|              OT|    WB|   N3A|             D19|          802|     WM32|         N15|      9122004568|   9122004568|      E13| SK06|        1797|6827603996|  27.10.2017|  916002812|      5307519917|27-10-2017|  WB23D5596|Road|36.0|0.016| 0.0|
|              OT|    WB

In [0]:
#In reverse order
reverse_df = v_dataset.sort((dataset.QTY).desc()).show()

+----------------+------+------+----------------+-------------+---------+------------+----------------+-------------+---------+-----+------------+----------+------------+-----------+----------------+----------+--------------------+----+-----+-----+---+
|Type_of_Customer|Region|SOffie|Regioal_S_Office|District_Code|City_Code|Cluster_Code|Distributor_Code|Customer_Code|WareHouse|STLOC|Product_Code|       Ref|Invoice_Date|Transporter|Delivery Challan|  Del_Date|          Truck_Code|MODE| Dist|  QTY|MRP|
+----------------+------+------+----------------+-------------+---------+------------+----------------+-------------+---------+-----+------------+----------+------------+-----------+----------------+----------+--------------------+----+-----+-----+---+
|              DC|    BR|   N5C|             D21|          209|     B636|         ZNR|      9124151043|   9124151043|      E06| B132|        1799|6826263781|  27.04.2017|  914501906|      5307135274|27-04-2017|    15 MULTIPLE NBOX|Rail|966.0

PERFORMING SQL QUERIES ON THE DATAFRAME

In [0]:
dataset.show()

+---+----------------+------+----------------+------------+----+------------+-----+----+----+------+
|_c0|Type of Customer|Region|Regioal S Office|Cluster Code|MODE|Product Code| Dist| QTY| MRP| Sales|
+---+----------------+------+----------------+------------+----+------------+-----+----+----+------+
|  0|               1|    12|               7|         283|   1|        1788|  2.0|10.0|72.0| 720.0|
|  1|               1|    26|               3|          22|   1|        1788|207.0|16.0|68.6|1097.6|
|  2|               1|    26|               3|         799|   1|        1788|482.0|21.0|69.4|1457.4|
|  3|               1|    12|               7|         278|   1|        1788|135.0|10.0|74.0| 740.0|
|  4|               1|     6|               4|         330|   1|        1788| 51.0|18.0|45.0| 810.0|
|  5|               1|    10|               1|         481|   1|        1792|211.0|12.0|49.6| 595.2|
|  6|               1|    10|               1|         481|   1|        9813|211.0| 5.0|54.

In [0]:
dataset = dataset.drop('_c0')

In [0]:
data = dataset.withColumnRenamed("Type of Customer", "Type_of_Customer").withColumnRenamed("Regioal S Office", "Regioal_S_Office").withColumnRenamed("Cluster Code", "Cluster_Code").withColumnRenamed("Product Code", "Product_Code")

In [0]:
data.registerTempTable('big_data')
#SQL queries can be performed on this table

In [0]:
#from pyspark import SparkContext
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)


In [0]:
sqlContext.sql('select Cluster_Code from big_data').show()

+------------+
|Cluster_Code|
+------------+
|         283|
|          22|
|         799|
|         278|
|         330|
|         481|
|         481|
|         438|
|         506|
|         539|
|         539|
|         602|
|         602|
|         118|
|         298|
|         303|
|         521|
|         555|
|          72|
|         132|
+------------+
only showing top 20 rows



In [0]:
#Checking the minimum sales
sqlContext.sql('select max(Sales) from big_data').show()

+-----------------+
|       max(Sales)|
+-----------------+
|47836.92000000001|
+-----------------+



In [0]:
#select all the columns where the mrp is 72 bucks
sqlContext.sql('select * from big_data where Product_Code == 3079') .show()

+----------------+------+----------------+------------+----+------------+-----+----+----+-----------------+
|Type_of_Customer|Region|Regioal_S_Office|Cluster_Code|MODE|Product_Code| Dist| QTY| MRP|            Sales|
+----------------+------+----------------+------------+----+------------+-----+----+----+-----------------+
|               1|    26|               3|         555|   1|        3079|161.0|16.0|74.2|           1187.2|
|               1|    26|               3|         555|   1|        3079|114.0|16.0|74.2|           1187.2|
|               1|    26|               3|         493|   1|        3079|125.0| 9.0|71.6|            644.4|
|               1|    26|               3|         555|   1|        3079|185.0| 9.0|74.2|667.8000000000002|
|               1|    26|               3|         555|   1|        3079|185.0| 7.0|74.2|            519.4|
|               1|    26|               3|         555|   1|        3079|161.0|16.0|74.2|           1187.2|
|               1|    26|   

In [0]:
#Nested queries
sqlContext.sql('select * from big_data where QTY IN (select min(QTY) from big_data)').show()

+----------------+------+----------------+------------+----+------------+-----+----+----+------------------+
|Type_of_Customer|Region|Regioal_S_Office|Cluster_Code|MODE|Product_Code| Dist| QTY| MRP|             Sales|
+----------------+------+----------------+------------+----+------------+-----+----+----+------------------+
|               1|     4|               5|         596|   1|        3079|770.0|0.05|75.6|              3.78|
|               1|     4|               5|         596|   1|        7457|770.0|0.05|72.2|              3.61|
|               1|    20|               7|         259|   1|        1788| 80.0|0.05|72.0|               3.6|
|               1|    20|               7|         259|   1|        1788| 80.0|0.05|69.0|              3.45|
|               1|    21|               8|         312|   1|        1788| 52.0|0.05|56.0|2.8000000000000003|
|               1|    11|               8|         305|   1|        1788| 17.0|0.05|63.0|3.1500000000000004|
|               1| 

In [0]:
""" MODEL FITTING """ 
#Checking the datatypes of columns
data.printSchema()

root
 |-- Type_of_Customer: integer (nullable = true)
 |-- Region: integer (nullable = true)
 |-- Regioal_S_Office: integer (nullable = true)
 |-- Cluster_Code: integer (nullable = true)
 |-- MODE: integer (nullable = true)
 |-- Product_Code: integer (nullable = true)
 |-- Dist: double (nullable = true)
 |-- QTY: double (nullable = true)
 |-- MRP: double (nullable = true)
 |-- Sales: double (nullable = true)



In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
featureassembler=VectorAssembler(inputCols=['Type_of_Customer', 'Region', 'Regioal_S_Office', 'Cluster_Code', 'MODE', 'Product_Code', 'Dist', 'QTY', 'MRP'],outputCol = "Independent Features")

In [0]:
output = featureassembler.transform(data)

In [0]:
output.select("Independent Features").show()

+--------------------+
|Independent Features|
+--------------------+
|[1.0,12.0,7.0,283...|
|[1.0,26.0,3.0,22....|
|[1.0,26.0,3.0,799...|
|[1.0,12.0,7.0,278...|
|[1.0,6.0,4.0,330....|
|[1.0,10.0,1.0,481...|
|[1.0,10.0,1.0,481...|
|[1.0,10.0,1.0,438...|
|[1.0,26.0,3.0,506...|
|[1.0,21.0,8.0,539...|
|[1.0,21.0,8.0,539...|
|[1.0,10.0,1.0,602...|
|[1.0,10.0,1.0,602...|
|[1.0,12.0,7.0,118...|
|[1.0,12.0,7.0,298...|
|[1.0,12.0,7.0,303...|
|[1.0,12.0,7.0,521...|
|[1.0,26.0,3.0,555...|
|[1.0,26.0,3.0,72....|
|[1.0,10.0,1.0,132...|
+--------------------+
only showing top 20 rows



In [0]:
data = output.select("Independent Features", "Sales")

In [0]:
train_data, test_data = data.randomSplit([0.7,0.3])

In [0]:
from pyspark.ml.regression import LinearRegression

regressor = LinearRegression(featuresCol='Independent Features', labelCol='Sales')
regressor = regressor.fit(train_data)

In [0]:
regressor.coefficients

DenseVector([-145.7082, -0.1013, -0.8337, -0.0237, -2366.8915, -0.0025, -0.0901, 68.1298, 10.678])

In [0]:
regressor.intercept

1815.4480584825212

In [0]:
pred_results = regressor.evaluate(test_data)

In [0]:
result = pred_results.predictions